# EMD algorithm - quick and dirty implementation

## Maria Inês Silva
## 08/01/2019

***

## Pseudo-code from the paper

The efficient EMD paper uses the following parameter setting:
* Tmin - the minimum length of analysis window (in data points).
* a - the SAX alphabet size.
* w - the number of SAX symbols in one BS.
* aw - the size of the analysis window, which is measured in the number of BSs.

### Algorithm EMD

1. Transform the original time series to PAA representation, using a slideing window of size `T_min`.
2. Transform the PAA reduced time series to SAX symbol sequence.
3. Transform the SAX symbol sequence to BS sequence.
4. Transform the BS sequence to Modified BS sequence.
5. Find all the motif candidates:
    1. Set the size of analysis window, W, to `T_min`.
    2. Extract all modified BS subsequences under the analysis window, by sliding the window from left to right. Find some DL pattern from these subsequences, if any. If there exists no DL pattern found and the window is now at the end of the Modified BS sequence, go to step 6.
    3. From the set of all pattern instances found in 5.B, establish the distance matrix for them (using DTW distance to calculate the distances between them). Call the procedure `Extract_Motif_Candidate` to find the motif candidate from the distance matrix, and calculate the MDL value of the candidate.
    4. Add the motif candidate to the result list along with its MDL value.
    5. Increase the size of the analysis window (i.e., Set W: = W + 1), go to 5.2
6. From the result list, find the motif candidate with the smallest MDL value. The found motif will be the returned result

### Algorithm `Extract_Motif_Candidate`

1. For each TSS in the distance matrix, identify all the other TSS which are similar to it (i.e., the distance between them is less than the threshold R).
2. Select as the pattern instances the TSSs which have the highest count of its similar subsequence.
3. Among the pattern instances selected in step 2, determine the instance which has the smallest sum of distances to all the other instances. This one is considered as the centre of the pattern (that means a motif candidate).

***

## Data and library imports

In [1]:
import numpy as np
import pandas as pd
import pickle
import os
import math
import time
from dtaidistance import dtw

In [2]:
cwd = os.getcwd()
data_folder = os.path.abspath(os.path.join(cwd, os.pardir, 'data'))
data_folder

'/Users/maria.silva/Documents/Tese/extendedMD/data'

In [3]:
ay = pd.read_csv(os.path.join(data_folder, 'ay.csv'), names=['ay'])
az = pd.read_csv(os.path.join(data_folder, 'az.csv'), names=['az'])

data = ay.assign(az = az['az'])
data.head()

,ay,az
0,-0.034,0.013
1,-0.005,0.003
2,0.006,0.010
3,0.004,-0.012
4,-0.012,-0.027


## 1. PCA

**input:** data (pandas dataframe where each column is a timeseries.Number of columns is the number of variable to apply pca)

**output:** numpy array that represents the 1-dimensional time series resulting from the PCA.

In [4]:
def extract_pca_ts(multi_dim_ts):
    # compute vector with the mean of each time series
    means_vec = data.agg('mean').values
    # compute the covariance matrix of the multi-dim time-series data
    cov_mat = multi_dim_ts.cov().values
    # extract eigenvalues and eigenvectors (the PCs) of the covariance matrix
    e_val, e_vec = np.linalg.eigh(cov_mat)
    # get the eigenvector with the highest eigenvalue (i.e. the 1st PC)
    pc1_vec = e_vec[np.argmax(e_val)]
    # compute the 1-dim time series as the data's projection on the 1st PC
    ts_1d = np.dot((multi_dim_ts.values - means_vec), pc1_vec)
    return ts_1d

In [5]:
ts_1d = extract_pca_ts(data)
ts_1d

array([ 0.03494258,  0.00693807, -0.00463669, ..., -0.04965133,
       -0.07109884, -0.07675976])

## 2. SAX

Code based on the function `sax_via_window` from `saxpy.sax`.

**input:** 1-d time series, sliding window size, PAA representation size, alphabet size and z-normalization threshold (??)

**output:** list of sax words (one word per sliding window)

In [6]:
from saxpy.znorm import znorm
from saxpy.paa import paa
from saxpy.alphabet import cuts_for_asize
from saxpy.sax import ts_to_string

def extract_sax_sequence(ts, win_size, paa_size, alphabet_size=3, z_threshold=0.01):
    # initialize list with sax sequence
    sax_sequence = []
    # get the cuts thresholds based on the gaussian distribution
    cuts = cuts_for_asize(alphabet_size)
    for t in range(0, len(ts) - win_size + 1):
        # define the current window
        ts_win = ts[t:(t+win_size)]
        # normalize the window
        ts_win_normalized = znorm(ts_win, z_threshold)
        # compute PAA representation of normalized window
        paa_rep = paa(ts_win_normalized, paa_size)
        # compute sax representation of PAA representation
        sax_word = ts_to_string(paa_rep, cuts)
        # append sax word to sax sequence list
        sax_sequence.append(sax_word)
    return sax_sequence

In [7]:
win_size=9
paa_size=3
alphabet_size=3

sax_sequence = extract_sax_sequence(ts_1d, win_size, paa_size, alphabet_size)
np.unique(sax_sequence, return_counts=True)

(array(['aac', 'abb', 'abc', 'aca', 'acb', 'acc', 'bab', 'bac', 'bba',
        'bbb', 'bbc', 'bca', 'bcb', 'caa', 'cab', 'cac', 'cba', 'cbb',
        'cca'], dtype='<U3'),
 array([ 291,  158, 1255,  122,  752,  244,  120,  725,  145, 1294,  126,
         736,  103,  263,  794,  100, 1368,  115,  267]))

## 3. Extract modified BS-sequence

**input:** sax sequence

**output:** a list with the modified bs sequence and a list with the lenght of each bs in the sequence (if there were two sax words together, then the lenght of the result bs sequence is 2)

In [8]:
def extract_modified_bs_sequence(sax_sequence):
    # initialize the lists to save the bs and their lenghts
    bs_seq = []
    bs_len = []
    # initialize the bs lenght counter
    curr_len = 1
    for i in range(len(sax_sequence)):
        # set the current bs element
        curr_bs = sax_sequence[i]
        # set the next bs element
        if i<len(sax_sequence)-1:
            next_bs = sax_sequence[i+1]
        else: # if the current element is the last, then thre's no "next_bs"
            next_bs = ''
        # test if the current bs is equal to the next bs
        if curr_bs==next_bs:
            # if yes, add 1 to the current lenght counter
            curr_len = curr_len + 1
        else:
            # if no, save the bs and its lenght in the corresponding lists
            bs_seq.append(curr_bs)
            bs_len.append(curr_len)
            # and initialize the lenght counter
            curr_len = 1
    return bs_seq, bs_len

def generate_bs_pointers(bs_len, bs_size):
    bs_pointers = []
    start_pointer = 0
    for bs_len_item in bs_len:
        end_pointer = start_pointer + bs_size + bs_len_item - 1
        pointer_list = list(range(start_pointer, end_pointer))
        bs_pointers.append(pointer_list)
        start_pointer = start_pointer + bs_len_item
    return bs_pointers

In [9]:
bs_seq, bs_len = extract_modified_bs_sequence(sax_sequence)
bs_point = generate_bs_pointers(bs_len, win_size)

## 4. Get list of all BS subsequences with a fixed BS size

**input:** bs_seq, bs_point and subseq_size

**output:** subseq_bs_list and subseq_point_list

In [10]:
def get_bs_subsequences_list(bs_seq, bs_point, subseq_size):
    # initialize lists to save the subsequences
    subseq_bs_list = []
    subseq_point_list = []
    for i in range(len(bs_seq) - subseq_size + 1):
        # extract the bs subsequence and append it to the list
        subseq_bs = bs_seq[i:(i+subseq_size)]
        subseq_bs_list.append(subseq_bs)
        # extract the pointers of the bs subsequence and append it to the list
        subseq_point = list(set(np.concatenate(bs_point[i:(i+subseq_size)])))
        subseq_point_list.append(subseq_point)
    return subseq_bs_list, subseq_point_list

In [11]:
subseq_size = 3

subseq_bs_list, subseq_point_list = get_bs_subsequences_list(bs_seq, bs_point, subseq_size)
print(len(subseq_bs_list))
subseq_bs_set = [list(item) for item in set(tuple(row) for row in subseq_bs_list)]
print(len(subseq_bs_set))

5933
1073


## 5. Extract all time series subsequences in a BS pattern

**input:** ts, subseq_bs_list, subseq_len_list, subseq_point_list and pattern

**output:** pattern_ts_list, pattern_len_listand  pattern_point_list

In [12]:
def get_all_subsequences_in_pattern(ts, subseq_bs_list, subseq_point_list, pattern):
    # initialize list to save the subsequences that bellong to the pattern
    pattern_ts_list = []
    pattern_pos_list = []
    # for each bs subsequence:
    for i in range(len(subseq_bs_list)):
        # if the subsequence bellongs to the pattern:
        if subseq_bs_list[i] == pattern:
            # save the position of the subsequence in the list
            pattern_pos_list.append(i)
            # extract the initial ts data of that subsequence and append it to the list
            pattern_ts = ts[subseq_point_list[i]]
            pattern_ts_list.append(pattern_ts)
    return pattern_ts_list, pattern_pos_list

In [13]:
pattern = subseq_bs_list[2]
pattern_ts_list, pattern_pos_list = get_all_subsequences_in_pattern(ts_1d, subseq_bs_list, subseq_point_list, pattern)
print(pattern_ts_list)
print(pattern_pos_list)

[array([-0.00463669, -0.00070325,  0.01655791, -0.00079149, -0.01538003,
        0.00268896,  0.01421275,  0.07041003,  0.04241922, -0.00952895,
       -0.0225292 , -0.01338783,  0.00155366]), array([-0.04492185, -0.03451967, -0.03950017, -0.04705325, -0.04105118,
       -0.03914721, -0.01181134, -0.02663911, -0.04600619, -0.05321812,
       -0.0562319 , -0.05247494])]
[2, 5505]


## 6. Compute DTW distance matrix

**input:** ts_list and R (max distance)

**output:** dist_matrix

In [14]:
def compute_dtw_dist_mat(ts_list, R=None):
    dist_matrix_vec = dtw.distance_matrix(ts_list, parallel=True, max_dist=R)
    dist_matrix = np.triu(dist_matrix_vec) + np.triu(dist_matrix_vec).T
    np.fill_diagonal(dist_matrix, 0)
    return dist_matrix

In [15]:
dist_mat = compute_dtw_dist_mat(pattern_ts_list)
dist_mat

array([[0.        , 0.16111923],
       [0.16111923, 0.        ]])

## 7. Find center and final members of a pattern

This step implements the distance constrain. And the overlapping contrain?

**input:**

**output:** 

In [16]:
def find_index_of_pattern_center_and_members(dist_mat, pattern_pos_list, R):
    # initialize lists to save row counters
    count_subseq_list = []
    sum_dist_list = []
    # for each row in the dist mat:
    for row in dist_mat:
        # compute the count of subseq with a distance < R and append it to the list
        count_subseq = sum(row < R)
        count_subseq_list.append(count_subseq)
        # sum the distances of those subseq and append it to the list
        sum_dist = sum(row[row < R])
        sum_dist_list.append(sum_dist)
    # convert lists to arrays (needed for the argmin)
    count_subseq_list = np.array(count_subseq_list)
    sum_dist_list = np.array(sum_dist_list)
    # find the maximum count of subseq
    max_num = max(count_subseq_list)
    # find the center, i.e., subseq with max count and min sum of dist
    center_index = np.argmin(sum_dist_list[count_subseq_list == max_num])
    # extract the dist row of the center
    center_row = dist_mat[center_index]
    # extract the members
    members_index = [index for index,value in enumerate(center_row) if value < R]
    #extract the center and members position in the subseq list
    center_pos = pattern_pos_list[center_index]
    members_pos = [pattern_pos_list[i] for i in members_index]
    return center_pos, members_pos

In [17]:
R = 30
center_pos, members_pos = find_index_of_pattern_center_and_members(dist_mat, pattern_pos_list, R)
print(center_pos)
print(members_pos)

2
[2, 5505]


## 8. Compute MDL

**input:**

**output:** 

In [18]:
def break_bs_len_seq(bs_len, break_points, subseq_size):
    bs_len_break = []
    if break_points[0] > 0:
        first_seq = bs_len[0:break_points[0]]
        bs_len_break.append(first_seq)
    for i in range(len(break_points)-1):
        pattern_seq = bs_len[break_points[i]:break_points[i]+subseq_size]
        bs_len_break.append(pattern_seq)
        next_seq = bs_len[break_points[i]+subseq_size:break_points[i+1]]
        bs_len_break.append(next_seq)
    final_pattern_seq = bs_len[break_points[-1]:break_points[-1]+subseq_size]
    bs_len_break.append(final_pattern_seq)
    if break_points[-1]+subseq_size < len(bs_len)-1:
        final_seq = bs_len[break_points[-1]+subseq_size:]
        bs_len_break.append(final_seq)
    return bs_len_break

In [19]:
bs_segmentation_len_list = break_bs_len_seq(bs_len, members_pos, subseq_size)
print([len(seq) for seq in bs_segmentation_len_list])

[2, 3, 5500, 3, 427]


In [20]:
def compute_pattern_mdl(seg_len_list):
    par_cost_list = []
    data_cost_list = []
    for seg in seg_len_list:
        seg_len_sum = float(sum(seg))
        seg_par_cost = math.log2(seg_len_sum)
        par_cost_list.append(seg_par_cost)
        seg_data_cost_list = [-l*math.log2(l/seg_len_sum) for l in seg]
        seg_data_cost = sum(seg_data_cost_list)
        data_cost_list.append(seg_data_cost)
    par_cost = sum(par_cost_list)
    data_cost = sum(data_cost_list)
    seg_cost = len(seg_len_list)*math.log2(sum(np.concatenate(seg_len_list)))
    mdl_cost = par_cost + data_cost + seg_cost
    return mdl_cost

In [21]:
compute_pattern_mdl(bs_segmentation_len_list)

106644.49675378966

## Extact all motif candidates

5. Find all the motif candidates:
    1. Set the size of analysis window, W, to `T_min`.
    2. Extract all modified BS subsequences under the analysis window, by sliding the window from left to right. Find some DL pattern from these subsequences, if any. If there exists no DL pattern found and the window is now at the end of the Modified BS sequence, go to step 6.
    3. From the set of all pattern instances found in 5.B, establish the distance matrix for them (using DTW distance to calculate the distances between them). Call the procedure `Extract_Motif_Candidate` to find the motif candidate from the distance matrix, and calculate the MDL value of the candidate.
    4. Add the motif candidate to the result list along with its MDL value.
    5. Increase the size of the analysis window (i.e., Set W: = W + 1), go to 5.2
6. From the result list, find the motif candidate with the smallest MDL value. The found motif will be the returned result

**input:** ts, bs_seq, bs_point, R

**output:** motif_mdl, motif_pointers, 

In [22]:
def find_all_motif_candidates(ts, bs_seq, bs_len, bs_point, R):
    # initialize the output lists
    mdl_cost_list = []
    motif_point_list = []
    # initialize the size of the BS subsequences
    subseq_size = 1
    while True:
        subseq_bs_list, subseq_point_list = get_bs_subsequences_list(bs_seq, bs_point, subseq_size)
        subseq_bs_set = [list(item) for item in set(tuple(row) for row in subseq_bs_list)]
        if len(subseq_bs_list)==len(subseq_bs_set):
            break
        for pattern in subseq_bs_set:
            pattern_ts_list, pattern_pos_list = get_all_subsequences_in_pattern(ts, subseq_bs_list, subseq_point_list, pattern)
            dist_mat = compute_dtw_dist_mat(pattern_ts_list)
            # compute the center the the members of the motif candidate and
            # return their position in the BS subsequence
            center_pos, members_pos = find_index_of_pattern_center_and_members(dist_mat, pattern_pos_list, R)
            # get the segmentation of the BS subseq fo lenghts, based on the motif (the motif segmentation)
            bs_segmentation_len_list = break_bs_len_seq(bs_len, members_pos, subseq_size)
            # Exclude empty lists (happens when there's overlapping patterns) --------------------------------------------- NEED TO CORRECT THIS!!!!
            bs_segmentation_len_list = [item for item in bs_segmentation_len_list if len(item)>0]
            # compute the MDL of the motif segmentation and append it to the list
            mdl_cost = compute_pattern_mdl(bs_segmentation_len_list)
            mdl_cost_list.append(mdl_cost)
            # extract the motif pointers (i.e. indices of the ts where all the motif members are located)
            # and append it to the list
            motif_point = [subseq_point_list[i] for i in members_pos]
            motif_point_list.append(motif_point)
        print('motif candidates of size {} successfully extracted'.format(subseq_size))
        # go to the next subsequence size
        subseq_size = subseq_size + 1
    return mdl_cost_list, motif_point_list

In [23]:
start_time = time.time()

R = 1
mdl_cost_list, motif_point_list = find_all_motif_candidates(ts_1d, bs_seq, bs_len, bs_point, R)

pickle.dump(mdl_cost_list, open("mdl_cost.p", "wb"))
pickle.dump(motif_point_list, open("motif_point.p", "wb"))

print("--- %s seconds ---" % (time.time() - start_time))

motif candidates of size 1 successfully extracted
motif candidates of size 2 successfully extracted
motif candidates of size 3 successfully extracted
motif candidates of size 4 successfully extracted
motif candidates of size 5 successfully extracted
motif candidates of size 6 successfully extracted
motif candidates of size 7 successfully extracted
motif candidates of size 8 successfully extracted
motif candidates of size 9 successfully extracted
motif candidates of size 10 successfully extracted
motif candidates of size 11 successfully extracted
motif candidates of size 12 successfully extracted
motif candidates of size 13 successfully extracted
motif candidates of size 14 successfully extracted
motif candidates of size 15 successfully extracted
motif candidates of size 16 successfully extracted
--- 9410.520062208176 seconds ---


In [26]:
pd.Series(mdl_cost_list).describe()

count     69422.000000
mean     103076.067987
std        3847.449414
min       43997.594614
25%      101453.645894
50%      102592.228909
75%      105039.380489
max      110247.840514
dtype: float64

In [25]:
print(min(mdl_cost_list))
motif_index = np.argmin(mdl_cost_list)
motif_pointers = motif_point_list[motif_index]

43997.594614424896
